Check python version

change python version through menu: Runtime -> Change runtime type

choose python 3 and gpu

Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Install libraries

In [ ]:
%pip install -U PyYAML
%pip install neat-python torch
%pip install chainer
# donwgrade numpy
%pip install --upgrade numpy==1.25.2
# downgrade setuptools
%pip install "setuptools <65"

# might become necessary later
# pip install 'cupy-cuda111>=7.7.0,<8.0.0'

Pull code from github

In [ ]:
!rm -r illusion_evolver
!git clone https://github.com/LanaSina/illusion_evolver.git -b cleanup --recurse-submodules

In [ ]:
# use in case of weird colab encoding error on shell commands
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

print(locale.getpreferredencoding())


Download prediction model.

black and white model: https://doi.org/10.6084/m9.figshare.13280120

color model: https://figshare.com/articles/Sample_Weight_Model_Front_Psychol_15_March_2018_/11931222

Then drag and drop to the left of this screen, or save in your google drive and change the path below to your own drive.

In [ ]:
# color
!cp '/content/drive/My Drive/AI/prednet_private/models/fpsi_500000_20v.model' '.'

# black and white
#!cp '/content/drive/My Drive/AI/prednet_private/models/black_and_white/300000_wb.model' '.'

Generate illusions. The results will be saved to the "results" folder on the left. Stop the algorithm whenever you are satistified.


In [ ]:
!rm -r "results"
!rm -r "temp"

# argument '-s': Type of illusion. 0: Bands; 1: Circles (default=0)
# g: gradient. 0 = no gradient, 1 = with gradient
# color
%run 'illusion_evolver/generate_illusion.py' -m 'fpsi_500000_20v.model' -s 1 -o './results' -g 1

# black and white
# %run 'illusion_evolver/generate_illusion.py' -m '300000_wb.model' -s 1 -c 1 -o './results' --channels "1,16,32,64" -g 1

# black and white without gradient
# %run 'illusion_evolver/generate_illusion.py' -m '300000_wb.model' -s 1 -c 1 -o './results' --channels "1,16,32,64" -g 0

Dowload results by left-click and "save image as", or download whole folder.



In [ ]:
# download images
!zip -r "results.zip" "results"
from google.colab import files
files.download('results.zip')

Get the visual flow and fitness score of an image (no tiling)

In [ ]:
from illusion_evolver import fitness_calculator
import os
from PIL import Image

# resize
image_path = 'single-snake.jpg'
model_name = 'fpsi_500000_20v.model'
channels = [3,48,96,192]

if not os.path.exists("predictions/"):
    os.makedirs("predictions/")

image = Image.open(image_path).convert('RGB')
#resize with padding rather than squish
maxwidth = 160
maxheight = 120
wpercent = min(maxwidth/image.width, maxheight/image.height)
wsize = int((float(image.width) * float(wpercent)))
hsize = int((float(image.height) * float(wpercent)))
image = image.resize((wsize, hsize), Image.Resampling.LANCZOS)
#image.save("output.png")
# pad with white
result = Image.new(image.mode, (maxwidth, maxheight), (255, 255, 255))
left = (int)((maxwidth-wsize)/2)
top = (int)((maxheight-hsize)/2)
result.paste(image, (left, top))
result.save("resized_image.png")

# image = image.resize((160,120))
size = [maxwidth,maxheight]
# image.save("resized_image.png")

In [ ]:
vectors = fitness_calculator.get_vectors("resized_image.png", model_name, size[0], size[1])

score = fitness_calculator.calculate_fitness(structure = 1, vectors = vectors,
                                             image_path="resized_image.png",
                                             w = 160, h = 120
                                             )
print(score)

In [ ]:
from illusion_evolver.optical_flow.optical_flow import lucas_kanade
from illusion_evolver import fitness_calculator

results = lucas_kanade("resized_image.jpg", "predictions/0000000000.png", "flow/", save=True)

if results["vectors"]:
    original_vectors = np.asarray(results["vectors"])
else:
    original_vectors = [[0,0,-1000,0]]

final_score = -100
temp_index = -1
mean_score = 0
score = 0
index = 0
if(len(original_vectors[index])>0):
    # bonus
    score = score + 0.1
    ratio = plausibility_ratio(original_vectors[index])
    score_0 = ratio[0]
    good_vectors = ratio[1]

    if(len(good_vectors)>0):
        score = score + 0.1
        step = h/2
        y = 0
        count = 0
        score_2 = [None]*2
        while y<h:
            limit = [y, y+step]
            score_2[count] = direction_ratio(good_vectors, limit)
            y = y + step
            count = count + 1

        # bonus points
        if(score_2[0]*score_2[1]<0):
            # is the ideal number of vectors
            temp = 24 - len(good_vectors)
            if(temp==0):
                n_dist = 1
            else:
                n_dist = 1/temp*temp
            score = score + n_dist*(abs(score_2[0]) + abs(score_2[1]))/2
            mean_score = mean_score + score

print("fitness score", score)

In [ ]:
%run "illusion_evolver/optical_flow/optical_flow.py" "resized_image.jpg" "predictions/0000000000.png" -s 0 -l 1 -cc yellow -lc red -s 2 -l 2 -vs 60.0